# Practical Course 1

In [667]:
import sys
sys.path.append('../../rai/rai/ry')
import libry as ry
import numpy as np

In [668]:
class RaiRobot():
    
    def __init__(self, nodeName:str, fileName:str):
        self.C = ry.Config()
        self.D = self.C.view()
        self.C.clear()
        self.C.addFile(fileName)
        self.q_home = self.C.getJointState()
        self.q_zero = self.q_home * 0.
        self.B = self.C.operate(nodeName)
        self.B.sync(self.C)
        self.real = False
        self.B.sendToReal(self.real)
        self.IK = self.C.komo_IK()
        #self.C.addObject(name="ball", shape=ry.ST.sphere, size=[.1], pos=[.8,.8,1.5], color=[1,1,0])
    
    def getFrameNames(self)-> list:
        return self.C.getFrameNames()
    
    def moveToPosition(self, pose:np.ndarray, frameName:str):
        self.IK.clearObjectives
        self.IK.addObjective(type=ry.OT.none, feature=ry.FS.pose, frames=['l_shoulder_lift_joint', frameName], target=pose)
        self.IK.optimize()
        self.C.setFrameState(self.IK.getConfiguration(0))
        self.q = self.IK.getConfiguration(0)
        print(self.q)
        print(self.q.shape)
        self.B.move([self.C.getJointState()], [10.], False)
        
        
        #self.inverseKinematics([obj])
        
    def inverseKinematics(self, objectives):
        self.IK.clearObjectives()
        for objective in objectives:
            print(objective)
            self.IK.addObjective(**objective)
        self.IK.optimize()
    
    def goHome(self):
        self.B.move([self.q_home], [10.0], False)
    
    def sendToReal(self, val:bool):
        self.real = val
        self.B.sendToReal(val)

In [669]:
def reset(robot):
    robot.C = 0
    robot.D = 0
    robot.B = 0
    return RaiRobot('', '../../rai-robotModels/pr2/pr2.g')

In [670]:
robot = RaiRobot('', '../../rai-robotModels/baxter/baxter.g')

In [671]:
robot = reset(robot)

In [673]:
des_pose = getPose('pr2L') + [0,0,0.1,0,0,0,0]
print(des_pose)
robot.moveToPosition(des_pose, 'pr2L')


[ 0.03546437  0.10699348  1.12878805  0.06126077  0.366274    0.41119915
  0.83246966]
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.21205477 -0.0304469   0.         ...,  0.          0.         -0.02301222]
 [ 0.16210773 -0.02814628  1.0875232  ..., -0.0162721  -0.70691953
  -0.0162721 ]
 ..., 
 [ 0.67875837 -0.27246411  1.12785686 ..., -0.93077296  0.33985782
  -0.11955487]
 [ 0.18394318 -0.27764103  0.85627088 ..., -0.06486709  0.47088902
  -0.23783895]
 [ 0.18063704 -0.28018989  0.85009255 ..., -0.06486709  0.47088902
  -0.23783895]]
(164, 7)


In [ ]:
robot.gazeAt('baxterL', 'ball')

In [ ]:
def getIK(q):
    IK.clearObjectives()
    IK.addObjective(type=OT.eq, feature=FS.positionDiff, frames=['hand', 'ball'], target=[.1, .1, .1])
    IK.optimize()
    C.setFrameState( IK.getConfiguration(0) )
    

In [17]:
def getPose(name):
    return robot.C.getFrameState(name)
    #print('frame state: ', X0) 

In [9]:
getPose('baxterL')

array([ 0.58181738,  0.19057273,  1.12082012,  0.6067361 ,  0.02677854,
       -0.00911642, -0.79440008])

In [645]:
len(robot.C.getFrameNames())


164

array([ 0.68181738,  0.19057273,  1.12082012,  0.6067361 ,  0.02677854,
       -0.00911642, -0.79440008])

In [ ]:
F = robot.C.feature(ry.FS.position, ['baxterL'])
F.eval(robot.C)